In [1]:
import os
import random
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import cv2
import pandas as pd
import numpy as np
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm import tqdm  # For progress bar
from torch.utils.data import DataLoader, random_split

In [2]:
# Path to the image dataset directory
root_dir = "/data/NNDL/data/image"

# Create a dictionary to group images by car make
car_make_dict = {}

for dirpath, _, filenames in os.walk(root_dir):
    for file in filenames:
        if file.endswith(".jpg"):
            # Extract car make from the path (3rd-level directory)
            parts = dirpath.split(os.sep)
            car_make = parts[-3]  # The 3rd-level directory is the car make
            
            if car_make not in car_make_dict:
                car_make_dict[car_make] = []
            car_make_dict[car_make].append(os.path.join(dirpath, file))

# Function to create positive and negative pairs
def create_pairs(car_make_dict, num_pairs=10000):
    pairs = []
    makes = list(car_make_dict.keys())
    
    for _ in range(num_pairs):
        # Positive pair (same car make)
        make = random.choice(makes)
        if len(car_make_dict[make]) >= 2:
            img1, img2 = random.sample(car_make_dict[make], 2)
            pairs.append((img1, img2, 0))  # label 0 for similar
        
        # Negative pair (different car makes)
        make1, make2 = random.sample(makes, 2)
        img1 = random.choice(car_make_dict[make1])
        img2 = random.choice(car_make_dict[make2])
        pairs.append((img1, img2, 1))  # label 1 for dissimilar

    return pairs

# Generate 10,000 pairs (adjust num_pairs as needed)
pairs = create_pairs(car_make_dict, num_pairs=10000)

# Convert to a DataFrame for easier processing
pairs_df = pd.DataFrame(pairs, columns=["img1", "img2", "label"])

# Save the pairs to a CSV file
pairs_df.to_csv("car_pairs.csv", index=False)
print("Dataset pairs created and saved as car_pairs.csv!")

Dataset pairs created and saved as car_pairs.csv!


In [3]:

# Data augmentation and normalization for training and validation
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a fixed size
    transforms.RandomHorizontalFlip(),  # Random horizontal flip
    transforms.RandomRotation(15),  # Random rotation within 15 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Random changes in brightness, contrast, etc.
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Random crop and resize
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Random translation
    transforms.RandomGrayscale(p=0.1),  # Randomly convert to grayscale
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize to ImageNet stats
])

# Custom dataset class for loading and preprocessing image pairs
class CarPairDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.pairs_df = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.pairs_df)

    def __getitem__(self, idx):
        row = self.pairs_df.iloc[idx]
        img1_path = row["img1"]
        img2_path = row["img2"]
        label = torch.tensor(row["label"], dtype=torch.float32)

        img1 = self.load_image(img1_path)
        img2 = self.load_image(img2_path)

        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        return (img1, img2), label

    def load_image(self, image_path):
        """Load an image from a given path and convert it to PIL format."""
        img = cv2.imread(image_path)
        if img is None:
            print(f"Warning: Could not read {image_path}")
            img = np.zeros((299, 299, 3), dtype=np.uint8)  # Return a black image if loading fails
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        return Image.fromarray(img)

In [4]:
# Load the dataset
csv_file = "car_pairs.csv"  # Update this to your actual CSV file path
car_dataset = CarPairDataset(csv_file=csv_file, transform=transform)


# Total dataset size
total_size = len(car_dataset)

# Split sizes: 80% for training, 20% for validation
train_size = int(0.8 * total_size)
val_size = total_size - train_size

# Randomly split the dataset
train_dataset, val_dataset = random_split(car_dataset, [train_size, val_size])

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True , num_workers= 16, pin_memory=True, prefetch_factor=2, persistent_workers=True)
val_dataloader = DataLoader(val_dataset, batch_size=128, shuffle=False  , num_workers= 16, pin_memory=True, prefetch_factor=2, persistent_workers=True)

print(f"Train set size: {len(train_dataset)}, Validation set size: {len(val_dataset)}")

Train set size: 16000, Validation set size: 4000


In [4]:
import torch.nn.functional as F

# Use ResNet50 pretrained on ImageNet as the feature extractor
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        # Load pretrained ResNet50 and remove the last fully connected layer
        resnet50 = models.resnet50(pretrained=True)
        self.feature_extractor = nn.Sequential(*list(resnet50.children())[:-1])  # Remove the FC layer

        # Add a fully connected layer for feature comparison (optional)
        self.fc = nn.Sequential(
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Linear(512, 128)
        )

    def forward_once(self, x):
        """Pass the input through the feature extractor."""
        x = self.feature_extractor(x)
        x = x.view(x.size(0), -1)  # Flatten the output
        x = self.fc(x)
        return x

    def forward(self, input1, input2):
        """Compute embeddings for both inputs and return the distance."""
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2

# import torch
# import torch.nn as nn
# import torchvision.models as models
# import torchvision.transforms as transforms

# class SiameseNetwork(nn.Module):
#     def __init__(self):
#         super(SiameseNetwork, self).__init__()
#         # Load pretrained Inception v3 model
#         inception = models.inception_v3(pretrained=True, aux_logits=True)
        
#         # Remove the last fully connected layer
#         self.feature_extractor = nn.Sequential(
#             *list(inception.children())[:-1],  # Remove FC layer
#             nn.AdaptiveAvgPool2d((1, 1))  # Ensure consistent output shape
#         )
        
#         # Fully connected layers for embedding
#         self.fc = nn.Sequential(
#             nn.Linear(2048, 512),  # Inception's last layer outputs 2048 features
#             nn.ReLU(),
#             nn.Linear(512, 128)
#         )

#     def forward_once(self, x):
#         """Pass the input through the feature extractor."""
#         x = self.feature_extractor(x)  # Extract features
#         x = torch.flatten(x, 1)  # Flatten to (batch_size, 2048)
#         #x = self.fc(x)  # Pass through FC layers
#         return x
    
#     def forward(self, input1, input2):
#         """Compute embeddings for both inputs and return the distance."""
#         output1 = self.forward_once(input1)
#         output2 = self.forward_once(input2)
#         return output1, output2



# Define the contrastive loss function
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        """Compute contrastive loss."""
        euclidean_distance = torch.nn.functional.pairwise_distance(output1, output2)
        loss = torch.mean((1 - label) * torch.pow(euclidean_distance, 2) +
                          label * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))
        return loss
    

# # Define the Triplet Loss function
# class ContrastiveLoss(nn.Module):
#     def __init__(self, margin=1.0):
#         super(ContrastiveLoss, self).__init__()
#         self.margin = margin

#     def forward(self, anchor, positive, negative):
#         """Compute Triplet Loss"""
#         pos_dist = F.pairwise_distance(anchor, positive, p=2)
#         neg_dist = F.pairwise_distance(anchor, negative, p=2)
        
#         loss = torch.mean(torch.clamp(pos_dist - neg_dist + self.margin, min=0.0))
#         return loss


In [6]:
# Create the model, loss function, and optimizer
model = SiameseNetwork().cuda()  # Move model to GPU if available
criterion = ContrastiveLoss(margin=1.0)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)  # Reduce LR if no improvement

# Training loop with validation using the split dataset
num_epochs = 50
best_val_loss = float('inf')
early_stop_counter = 0
patience = 5  # Early stopping patience

/data/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/data/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/data/anaconda3/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [7]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    # Training phase
    for (img1, img2), labels in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        img1, img2, labels = img1.cuda(), img2.cuda(), labels.cuda()
        
        optimizer.zero_grad()
        output1, output2 = model(img1, img2)
        
        loss = criterion(output1, output2, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()

        # Compute accuracy
        similarity_scores = torch.nn.functional.pairwise_distance(output1, output2)
        predictions = (similarity_scores > 0.5).float()  # Assuming 0.5 as the threshold
        correct_predictions += (predictions == labels).sum().item()
        total_samples += labels.size(0)
    
    avg_train_loss = total_loss / len(train_dataloader)
    train_accuracy = correct_predictions / total_samples
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}, Training Accuracy: {train_accuracy:.4f}")

    # Validation phase
    model.eval()
    val_loss = 0.0
    correct_val_predictions = 0
    total_val_samples = 0

    with torch.no_grad():
        for (val_img1, val_img2), val_labels in val_dataloader:
            val_img1, val_img2, val_labels = val_img1.cuda(), val_img2.cuda(), val_labels.cuda()
            val_output1, val_output2 = model(val_img1, val_img2)
            val_loss += criterion(val_output1, val_output2, val_labels).item()

            # Compute validation accuracy
            val_similarity_scores = torch.nn.functional.pairwise_distance(val_output1, val_output2)
            val_predictions = (val_similarity_scores > 0.5).float()
            correct_val_predictions += (val_predictions == val_labels).sum().item()
            total_val_samples += val_labels.size(0)

    avg_val_loss = val_loss / len(val_dataloader)
    val_accuracy = correct_val_predictions / total_val_samples
    print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    # Early stopping and model saving
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        early_stop_counter = 0
        torch.save(model.state_dict(), "make_verification_resnet50.pth")
        print("Best model saved!")
    else:
        early_stop_counter += 1
        print(f"Early stopping counter: {early_stop_counter}/{patience}")

    # Reduce learning rate if validation loss doesn't improve
    scheduler.step(avg_val_loss)
    
    if early_stop_counter >= patience:
        print("Early stopping triggered. Training stopped.")
        break


Epoch 1/50: 100%|██████████| 125/125 [01:39<00:00,  1.26it/s]

Epoch [1/50], Training Loss: 0.2182, Training Accuracy: 0.6501


Validation Loss: 0.1920, Validation Accuracy: 0.7020
Best model saved!


Epoch 2/50: 100%|██████████| 125/125 [01:31<00:00,  1.36it/s]


Epoch [2/50], Training Loss: 0.1777, Training Accuracy: 0.7330
Validation Loss: 0.1769, Validation Accuracy: 0.7365
Best model saved!


...

Epoch [16/50], Training Loss: 0.0653, Training Accuracy: 0.9361
Validation Loss: 0.1574, Validation Accuracy: 0.7732
Early stopping counter: 3/5


Epoch 17/50: 100%|██████████| 125/125 [01:32<00:00,  1.35it/s]


Epoch [17/50], Training Loss: 0.0611, Training Accuracy: 0.9466
Validation Loss: 0.1609, Validation Accuracy: 0.7662
Early stopping counter: 4/5


Epoch 18/50: 100%|██████████| 125/125 [01:31<00:00,  1.37it/s]


Epoch [18/50], Training Loss: 0.0494, Training Accuracy: 0.9627
Validation Loss: 0.1530, Validation Accuracy: 0.7815
Early stopping counter: 5/5
Early stopping triggered. Training stopped.


In [ ]:
# # Define the test dataset class
# class TestCarPairDataset(Dataset):
#     def __init__(self, txt_file, root_dir, transform=None):
#         self.pairs = []
#         self.root_dir = root_dir
#         self.transform = transform
        
#         # Read the text file and parse image pairs
#         with open(txt_file, "r") as file:
#             lines = file.readlines()
            
#         for line in lines:
#             img1_rel, img2_rel, label = line.strip().split()
#             img1_path = os.path.join(root_dir, img1_rel)
#             img2_path = os.path.join(root_dir, img2_rel)
#             label = 1 - int(label)  # Swap labels (1 -> 0, 0 -> 1)
#             self.pairs.append((img1_path, img2_path, label))

#     def __len__(self):
#         return len(self.pairs)

#     def __getitem__(self, idx):
#         img1_path, img2_path, label = self.pairs[idx]
#         label = torch.tensor(label, dtype=torch.float32)
        
#         img1 = self.load_image(img1_path)
#         img2 = self.load_image(img2_path)
        
#         if self.transform:
#             img1 = self.transform(img1)
#             img2 = self.transform(img2)
        
#         return (img1, img2), label

#     def load_image(self, image_path):
#         """Load an image from the given path and convert it to PIL format."""
#         img = cv2.imread(image_path)
#         if img is None:
#             print(f"Warning: Could not read {image_path}")
#             img = np.zeros((224, 224, 3), dtype=np.uint8)  # Return a black image if loading fails
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
#         return Image.fromarray(img)

# # Define the same transformation used during training
# test_transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])

In [2]:
# Define the test dataset class
class TestCarPairDataset(Dataset):
    def __init__(self, txt_file, root_dir, transform=None):
        self.pairs = []
        self.root_dir = root_dir
        self.transform = transform
        
        # Read the text file and parse image pairs
        with open(txt_file, "r") as file:
            lines = file.readlines()
            
        for line in lines:
            img1_rel, img2_rel, _ = line.strip().split()
            img1_path = os.path.join(root_dir, img1_rel)
            img2_path = os.path.join(root_dir, img2_rel)
            
            # Extract the first directory (number) from each image path
            id1 = img1_rel.split("/")[0]
            id2 = img2_rel.split("/")[0]
            # If the first directory is the same, label is 0; otherwise, label is 1
            label = 0 if id1 == id2 else 1
            
            self.pairs.append((img1_path, img2_path, label))

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        img1_path, img2_path, label = self.pairs[idx]
        label = torch.tensor(label, dtype=torch.float32)
        
        img1 = self.load_image(img1_path)
        img2 = self.load_image(img2_path)
        
        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
        
        return (img1, img2), label

    def load_image(self, image_path):
        """Load an image from the given path and convert it to PIL format."""
        img = cv2.imread(image_path)
        if img is None:
            print(f"Warning: Could not read {image_path}")
            img = np.zeros((224, 224, 3), dtype=np.uint8)  # Return a black image if loading fails
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        return Image.fromarray(img)

# Define the same transformation used during training
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [5]:
# Load the trained model
model = SiameseNetwork().cuda()
model.load_state_dict(torch.load("make_verification_resnet50.pth"))
model.eval()


/data/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/data/anaconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/tmp/ipykernel_837388/240913420.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models fo

SiameseNetwork(
  (feature_extractor): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
         

In [6]:
# Path to the test dataset text file
test_txt_file = "/data/NNDL/data/train_test_split/verification/verification_pairs_easy.txt"
test_root_dir = "/data/NNDL/data/image"

# Create the test dataset and DataLoader
test_dataset = TestCarPairDataset(txt_file=test_txt_file, root_dir=test_root_dir, transform=test_transform)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=8, pin_memory=True)

### Easy 

In [7]:
# Test the model
correct_predictions = 0
total_samples = 0

with torch.no_grad():
    for (img1, img2), labels in test_dataloader:
        img1, img2, labels = img1.cuda(), img2.cuda(), labels.cuda()
        output1, output2 = model(img1, img2)
        
        similarity_scores = torch.nn.functional.pairwise_distance(output1, output2)
        predictions = (similarity_scores > 0.5).float()  # Using 0.5 as the threshold
        
        correct_predictions += (predictions == labels).sum().item()
        total_samples += labels.size(0)

# Calculate and print accuracy
test_accuracy = correct_predictions / total_samples
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.8831


### Medium

In [8]:
# Path to the test dataset text file
test_txt_file = "/data/NNDL/data/train_test_split/verification/verification_pairs_medium.txt"
test_root_dir = "/data/NNDL/data/image"

# Create the test dataset and DataLoader
test_dataset = TestCarPairDataset(txt_file=test_txt_file, root_dir=test_root_dir, transform=test_transform)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=8, pin_memory=True)

In [9]:
# Test the model
correct_predictions = 0
total_samples = 0

with torch.no_grad():
    for (img1, img2), labels in test_dataloader:
        img1, img2, labels = img1.cuda(), img2.cuda(), labels.cuda()
        output1, output2 = model(img1, img2)
        
        similarity_scores = torch.nn.functional.pairwise_distance(output1, output2)
        predictions = (similarity_scores > 0.5).float()  # Using 0.5 as the threshold
        
        correct_predictions += (predictions == labels).sum().item()
        total_samples += labels.size(0)

# Calculate and print accuracy
test_accuracy = correct_predictions / total_samples
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.8742


### Hard 

In [10]:
# Path to the test dataset text file
test_txt_file = "/data/NNDL/data/train_test_split/verification/verification_pairs_hard.txt"
test_root_dir = "/data/NNDL/data/image"

# Create the test dataset and DataLoader
test_dataset = TestCarPairDataset(txt_file=test_txt_file, root_dir=test_root_dir, transform=test_transform)
test_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=8, pin_memory=True)

In [11]:
# Test the model
correct_predictions = 0
total_samples = 0

with torch.no_grad():
    for (img1, img2), labels in test_dataloader:
        img1, img2, labels = img1.cuda(), img2.cuda(), labels.cuda()
        output1, output2 = model(img1, img2)
        
        similarity_scores = torch.nn.functional.pairwise_distance(output1, output2)
        predictions = (similarity_scores > 0.5).float()  # Using 0.5 as the threshold
        
        correct_predictions += (predictions == labels).sum().item()
        total_samples += labels.size(0)

# Calculate and print accuracy
test_accuracy = correct_predictions / total_samples
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 0.5618
